In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
file_name = './Result1(20days_RBF)/rst_2018-01-01_2023-05-10.csv'
#file_name = './Exp1new(20days_RBF)/rst_2018-01-01_2023-06-10_check.csv'
mydf = pd.read_csv(file_name)
mydf['Date'] = pd.to_datetime(mydf['Date'])
print(mydf)

In [ ]:
# グラフのレイアウトを設定
fig, axs = plt.subplots(3, 2, figsize=(10, 15))

# 指定した年の範囲
years = range(2018, 2024)

for i, year in enumerate(years):
    # 年ごとのデータを抽出
    mydf_year = mydf[mydf['Date'].dt.year == year]

    # 散布図を描く
    axs[i//2, i%2].scatter(mydf_year['Actual'], mydf_year['Predict'])

    # 45度線を描く
    axs[i//2, i%2].plot([mydf_year['Actual'].min(), mydf_year['Actual'].max()], [mydf_year['Actual'].min(), mydf_year['Actual'].max()], color='red')

    # グラフのタイトルと軸ラベルを設定
    axs[i//2, i%2].set_title(f'Scatter plot of Actual and Prediction in {year}')
    axs[i//2, i%2].set_xlabel('Actual')
    axs[i//2, i%2].set_ylabel('Prediction')
    axs[i//2, i%2].grid(True)
# グラフを表示
plt.tight_layout()
plt.show()

In [ ]:
# グラフのレイアウトを設定
fig, axs = plt.subplots(3, 2, figsize=(10, 15))

# 指定した年の範囲
years = range(2018, 2024)

for i, year in enumerate(years):
    # 年ごとのデータを抽出
    mydf_year = mydf[mydf['Date'].dt.year == year]

    # 散布図を描く
    axs[i//2, i%2].scatter(mydf_year['ProbAboveZero'], mydf_year['Predict'])
    axs[i//2, i%2].scatter(mydf_year['ProbAboveZero'], mydf_year['Actual'])

    # 45度線を描く
    axs[i//2, i%2].plot([0, 1], [0, 0], color='red')

    # グラフのタイトルと軸ラベルを設定
    axs[i//2, i%2].set_title(f'Prob Above Zero and Prediction and Actual in {year}')
    axs[i//2, i%2].set_xlabel('Probability Above Zero')
    axs[i//2, i%2].set_ylabel('Prediction/Actual')
    axs[i//2, i%2].grid(True)
# グラフを表示
plt.tight_layout()
plt.show()

In [ ]:
# グラフのレイアウトを設定
fig, axs = plt.subplots(3, 2, figsize=(10, 15))

# 指定した年の範囲
years = range(2018, 2024)

for i, year in enumerate(years):
    # 年ごとのデータを抽出
    mydf_year = mydf[mydf['Date'].dt.year == year]

    # 散布図を描く
    axs[i//2, i%2].plot(mydf_year['Date'], mydf_year['Predict'], linestyle='-', marker='.')
    axs[i//2, i%2].plot(mydf_year['Date'], mydf_year['Actual'], linestyle='-', marker='.')

    # グラフのタイトルと軸ラベルを設定
    axs[i//2, i%2].set_title(f'Prediction and Actual in {year}')
    axs[i//2, i%2].set_xlabel('date')
    axs[i//2, i%2].set_ylabel('Prediction/Actual')
    axs[i//2, i%2].grid(True)
# グラフを表示
plt.tight_layout()
plt.show()